In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from pprint import pprint
from scipy import stats
import string
import itertools as it
import timeit
import datetime 
import math
from operator import itemgetter


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
wpath = 'D:\\Kimon\\Documents\\Quantos-new\\z90-030\\Python code'
os.chdir(wpath)
current_time = datetime.datetime.now() 
outfiles_s = str(current_time.month)+ '_' + str(current_time.day) + \
                                      '_'+str(current_time.hour) + \
                                      '_'+str(current_time.minute)

KAPPA = 3 ; H = 3
flog = open(wpath + '\\log.txt', "w") 
FILE = "declaration_remuneration_2020_07_16_04_00.csv" ##"newdata2_del.csv"
ALPHA = 0.05
V1 = 0.99 ; V2 = 0.999
DROP_VARS = ['ligne_identifiant','remu_convention_liee'] ## or an empty list
DROP_VARS_PERC = 0.2

In [ ]:
def anonymise(inp, k): 
  ## perform heuristic k-anonymization on a dataset inp
 
    N, M = inp.shape
    dataCode = np.zeros((N,M),dtype=int)
    for c in range(M): 
        print('c= ',c)
        t1_idx = np.argsort(inp[:,c])
        t1 = inp[t1_idx,c]
        kq = np.empty((N,),dtype = np.int)
        i = 0
        sPT = 1 ## counter for row i
        group = 1 ## current group
        kq[0] = group ## assignment of row i
        n = len(t1)
        while i < n-1:
        #for i in range(1,n-1):    
            i += 1
            sPT += 1
            ## initialize new group if new value met and
            ## capacity of group is exceeded 
            if (sPT > k) and (t1[i]!=t1[i-1]): 
                group += 1
                sPT += 1
            kq[i] = group
    
        ## make last elements consistent
        if kq[n - 2] != kq[n - 3]:
            kq[n - 2] = kq[n - 3]
            kq[n - 1] = kq[n - 3]
        if kq[n - 1] != kq[n - 2]:
            kq[n - 1] = kq[n - 3]

        a3 = np.zeros(N,dtype = np.int)
        a3[t1_idx] = np.arange(N)
        dataCode[:,c] = kq[a3]
        
    return dataCode 


In [ ]:
def prob_re(inp,h):
    C = comb(len(inp.columns),3)
    res=[None]*C ; i=0 ; N = len(inp)
    start_time = timeit.default_timer()
    combins = [c for c  in it.combinations(inp.columns,3)]
    print(len(combins))
    for combin in combins:
        i += 1  
        #x = sum(rle_encoding(np.array(inp.groupby(list(comb),sort=True).size())))/N
        x = sum(pd.value_counts(inp.groupby(list(combin),sort=False).size(),
                            sort=False,dropna=False))/N
        #x = len(inp.groupby(list(comb),sort=False).nunique())/N
        #x = sum(np.bincount(inp.groupby(list(comb),sort=False).size()))/N
        #x = sum(Counter(inp.groupby(list(comb),sort=False).size()).values())/N
        if i % 100 == 0: print(i,x)
        res[i-1] = (combin,x)
    print('\nTime in prob_re(): {0:12.2f} secs\n'.format(timeit.default_timer() - start_time))
     
    
    return res

* Read FILE and fill missing values with "NA" 
* Check unique values and delete columns in DROP_VARS or if DROP_VARS is empty, delete variables with unique values $\geq$ DROP_VARS_PERC x number of rows 

In [ ]:
data = pd.read_csv(FILE,sep=';',dtype=object,low_memory=False)
data.fillna(value='NA', inplace=True)
data.isna().sum()

to_delete=list()
for col in data.columns:
    unis = len(pd.unique(data[col]))
    print('\n{0:30s} \nUnique values: {1:8d}'.format(col,len(pd.unique(data[col]))),end='')
    try:
        print('\n{0:30s} \nUnique values: {1:8d}'.format(col,len(pd.unique(data[col]))),end='',file=flog)
    except ValueError:
        pass
    if unis >= DROP_VARS_PERC*len(data): 
        print('     *****')
        try:
            print('     *****',file=flog)
        except ValueError:
            pass
        to_delete.append(col)
    else:
        print()

print('\nDeleting variables:\n')                
if len(DROP_VARS) > 0:        
    print(to_delete)
    print('*** Overriden:')
    print(DROP_VARS)
    data.drop(labels=DROP_VARS, axis=1, inplace=True)    
else:
    print(to_delete)
    data.drop(labels=to_delete, axis=1, inplace=True)    
data


In [ ]:
nms = data.columns
N,M = data.shape
np_data = np.zeros((N,M),dtype=int)
for c in range(len(data.columns)):
    np_data[:,c], _ = pd.factorize(data.iloc[:,c])
    #print('\n{0:2d} Unique values: {1:8d}'.format(c,len(pd.unique(np_data[:,c]))))
    try:
        print('\n{0:2d} Unique values: {1:8d}'.format(c,len(pd.unique(np_data[:,c]))),file=flog)
    except ValueError:
        pass



In [ ]:
pd.DataFrame(np_data).to_csv('np_data'+outfiles_s+'.csv',sep=';')
anon = anonymise(np_data, k=KAPPA)
anon = pd.DataFrame(anon,columns=data.columns)
anon.to_csv('anon'+outfiles_s+'.csv',sep=';')
anon

results = prob_re(anon,H)

In [ ]:
res_out = sorted(results,key=itemgetter(1),reverse=True)
topn = round(ALPHA*len(results))
res_out = res_out[:topn]
_, probs_out = zip(*res_out)

top_probs = pd.DataFrame(res_out,columns=['Combination','Probability'])  
top_probs

#_ = plt.hist(probs_out,bins=50)
#_ = plt.grid(True)
#plt.show()

flog.close()

_ = top_probs['Probability'].plot(kind='hist', bins=50, grid = True, figsize=(6,4))


In [ ]:
_, probs = zip(*results)
probs = [p for p in probs]

_ = plt.hist(probs,bins=50)
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.title('Histogram of probability of re-identification')
_ = plt.grid(True)
plt.show()

flog.close()

In [ ]:
def fit_GPD_PWM(excesses): 
    a0 = np.mean(excesses)
    x_s = sorted(excesses) ; n = len(x_s)
    kappas = np.arange(1,n+1) 
    product = (n-kappas)/(n-1)
    a1 = np.multiply(x_s,product)
    a1 = np.mean(a1)
    shape = 2 - a0/(a0 - 2 * a1)
    scale = max((2 * a0 * a1) / (a0 - 2 * a1),np.finfo(float).eps)
    return (shape,scale)

def id_danger(x ,v, alpha): 
    ## calculate the distribution of the (1-alpha) largest values, alpha = 0.95
    ## v = 0.99 or 0.999
  
    ## all values of the distribution
    ## in vector  x of length nx, sorted ascending in x_sort
    nx = len(x)
    x_sort = np.sort(x)    

    nx_ex = round(alpha * len(x)) ## will keep approx. nx_ex largest values
    id_u  = nx - nx_ex + 1 ## will drop id_u values
    u  = x_sort[id_u] ## cut-off
    
    id_ex = np.where(x >= u) ## which values of x to keep
    ## put these values in vector x_ex of length Nu
    x_ex = x[x >= u]
    Nu = len(x_ex)
    
    p_exceed = (x_ex > u).mean() # <- mean(x.ex > u) ## the proportion actually exceeding u
    
    ## fit to GPD and extract parameters xi (csi) and beta (shape and scale)
    
    (xi, beta) = fit_GPD_PWM(x_ex-u)
    ##print('shape= ',xi)
    ##print('scale= ',beta)

      
    ## the POT estimator - see p.7 top
    ## = the T(k,1-v) value: 
    ## 1% or 0.1% of size k quasi-identifiers will have a probability of re-identification greater than this value
    x_hat = u + beta / xi * ((nx / Nu * (1 - v)) ** (-xi) - 1)
  
    rl = dict()
    rl['sort'] = x_sort
    rl['value_extreme'] = x_ex
    rl['index_extreme'] = id_ex
    rl['p_exceed'] = p_exceed
    rl['obs'] = nx
    rl['Nb_extremes'] = Nu
    rl['threshold'] = u,
    rl['index.threshold'] = id_u,
    rl['shape'] = xi,
    rl['scale'] = beta,
    rl['POT'] = x_hat
    
    return rl
  
def id_danger_logit(x, v1, v2, alpha): 
  
    ## at level 1%
    res  = id_danger(x, v1, alpha)
    p99 = res['POT'] ## the temperature: 1% of size-k pseudo-identifiers
    ## will have a probability of re-identification greater than this value
    xi = res['shape']
    beta = res['scale']
    ##--------------------
  
    ## at level 0.1%
    res = id_danger(x, v2, alpha)
    p999 = res['POT']
    ## --------------------
  
    ## logit, 1%
    xx = np.log(x / (1 - x))
    res = id_danger(xx, v1, alpha)
    pi99 = res['POT']
    xi_logit =  res['shape']
    beta_logit =  res['scale']
    p99_logit = np.exp(pi99) / (1 + math.exp(pi99)) ## temperature
    ## --------------------
  
    ## logit, 0.1%
    res = id_danger(xx, v2, alpha)
    pi999 = res['POT']
    p999_logit = np.exp(pi999) / (1 + np.exp(pi999)) ## temperature
  
    rl = dict()
    rl['xi'] = xi
    rl['beta'] = beta    
    rl['xi_logit'] = xi_logit
    rl['beta_logit'] = beta_logit    
    rl['id_99'] = p99
    rl['id_999'] = p999
    rl['id_99_logit'] = p99_logit
    rl['id_999_logit'] = p999_logit

    return rl
    

In [ ]:
x = np.array(probs)

results = id_danger(x ,V1, ALPHA)
print(results)


tmp = pd.DataFrame(x_ex-u,columns=['Excess'])
_ = tmp['Excess'].plot(kind='hist', bins=50, grid = True, figsize=(6,4))

res = id_danger_logit(x, v1=V1, v2=V2, alpha=ALPHA) 

print('       {0:6.1f}% quantile of extreme values: {1:8.4f}'.format(100*V1,
              round(res['id_99'],4)))
print('       {0:6.1f}% quantile of extreme values: {1:8.4f}'.format(100*V2,
              round(res['id_999'],4)))
print('Logit: {0:6.1f}% quantile of extreme values: {1:8.4f}'.format(100*V1,
              round(res['id_99_logit'],4)))
print('Logit: {0:6.1f}% quantile of extreme values: {1:8.4f}'.format(100*V2,
              round(res['id_999_logit'],4)))


In [ ]:
#calculations <- function(x,v1,v2,alpha) {

#  tms <- id.danger(x, v1, alpha)
#  xi <- round(tms$shape, 4) ## xi: shape
#  beta <- round(tms$scale, 4) ## beta: beta
#  u <- tms$threshold ## u: threshold with v1,alpha
#  ex <- tms$value.extreme # extreme values
  
#  
#  logit = log(x / (1 - x))
#  tms2 <- id.danger(logit, v1, alpha)
#  xi2 <- round(tms2$shape, 4) ## xi2: shape - logit
#  beta2 <- round(tms2$scale, 4) ## beta2: scale - logit
#  xh1 <- tms2$POT
#  xhat1 <- exp(xh1) / (1 + exp(xh1))   # POT estimate with v1/logit

#  xh2 <- id.danger(logit, v2, alpha)$POT
#  xhat2 <- exp(xh2) / (1 + exp(xh2)) # POT estimate with v2/logit
  
#  xx <- seq(from = 0,to = 1,length = 1000)
#  z <- qgpd(xx, xi, u, beta) ## z: quantiles from qgpd()
#  y <- pgpd(z, xi, u, beta) ## y: probabilities from pgpd()
  
#  u1 <- id.danger(x, v1, 1 - v1)$threshold # u1: threshold with v1 and alpha = 1-v1
#  u2 <- id.danger(x, v2, 1 - v2)$threshold # u2: threshold with v2 and alpha = 1-v2
#  mep <- mean_excess_np(x) ## mean excess
  
  
  ## res <- list(ex=ex,y=y,z=z,xi=xi,beta=beta,xi2=xi2,beta2=beta2,u=u,u1=u1,u2=u2,
  ## mep=mep,xhat1=xhat1,xhat2=xhat2)
  ## ex: extreme values 
  ## y: probabilities from pgpd()
  ## z: quantiles from qgpd()
  ## xi: shape
  ## beta: beta
  ## xi2: shape - logit
  ## beta2: scale - logit
  ## u: threshold with v1,alpha 
  ## u1: threshold with v1 and alpha = 1-v1
  ## u2: threshold with v2 and alpha = 1-v2
  ## mep <- mean_excess_np(x) ## mean excess
  ## u1: threshold with v1 and alpha = 1-v1
  ## u2: threshold with v2 and alpha = 1-v2
  ## xhat1: POT estimate with v1/logit
  ## xhat2: POT estimate with v2/logit
#  
#  res <- list(ex=ex,y=y,z=z,xi=xi,beta=beta,xi2=xi2,beta2=beta2,u=u,u1=u1,u2=u2,
#              mep=mep,xhat1=xhat1,xhat2=xhat2)
#  return(res)


#}


